## History

In [11]:
import pandas as pd
datapath = "tuning/"
test_recommendations = pd.read_csv(datapath + "test_recommendations.csv")
train_recommendations = pd.read_csv(datapath + "train_recommendations.csv")
app_plots = pd.read_csv(datapath + "movies_with_desc.csv")
app_plots.columns, train_recommendations.columns, test_recommendations.columns

(Index(['app_id', 'title', 'tmdbId', 'imdbId', 'desc', 'genres'], dtype='object'),
 Index(['user_id', 'app_id', 'Rating', 'Timestamp'], dtype='object'),
 Index(['user_id', 'app_id', 'Rating', 'Timestamp'], dtype='object'))

In [12]:
df = pd.read_csv("tuning/test_recommendations.csv")

# 2. Calcola la popolarità normalizzata
# .value_counts(normalize=True) calcola automaticamente il rapporto (frequenza relativa)
popularity_normalized = df['app_id'].value_counts().reset_index()
popularity_normalized.columns = ['app_id', 'popularity']

# 3. Unisci la colonna di popolarità al DataFrame originale
# Viene unito in base alla colonna 'app_id'
df_with_popularity = pd.merge(df, popularity_normalized, on='app_id', how='left')

# Stampa il risultato per verifica, arrotondando la popolarità a 4 decimali per chiarezza
df_with_popularity['popularity'] = df_with_popularity['popularity'].round(4)
print(df_with_popularity)

        user_id  app_id  Rating   Timestamp  popularity
0          5950    2550       1   957180825          20
1          5565    2600       4  1000927588          57
2          3389    2167       3   967515284          86
3          5956     225       3   987038187          33
4           869    1673       3   975278781         126
...         ...     ...     ...         ...         ...
108018     5948    1198       5   957192748         236
108019     5124     176       3   962198943          29
108020     2181    1449       5   974669125          57
108021      702    3519       3   975563490          23
108022     1810    2206       3   974698728          23

[108023 rows x 5 columns]


In [13]:
# stampa la popolarità media
average_popularity = df_with_popularity['popularity'].mean()
print(f"Popolarità media: {1/average_popularity}")

Popolarità media: 0.011184182803403948


In [23]:
df_with_popularity.to_csv("tuning/test_recommendations_with_popularity.csv", index=False)

In [29]:
merged_train_df = pd.merge(train_recommendations, app_plots, on='app_id', how='inner')

# --- 3. Seleziona e Rinomina le Colonne Necessarie ---
# Selezioniamo solo le colonne che ci servono per il file finale.
# Rinominiamo 'genres' in 'genre' e 'rating' per corrispondere alla tua richiesta.
# Nota: ho corretto il tuo "raiting" in "rating".
history_train_data = merged_train_df[[
    'user_id',
    'title',
    'desc',
    'genres', # Selezioniamo la colonna originale
    'Rating'
]]

print("Creazione delle storie degli utenti...")
user_histories_train_df = (
    history_train_data.groupby('user_id')[['title', 'desc', 'genres', 'Rating']]
                .apply(lambda x: x.to_dict('records'))
                .reset_index(name='history')
)
merged_test_df = pd.merge(test_recommendations, app_plots, on='app_id', how='inner')
history_test_data = merged_test_df[[
    'user_id',
    'title',
    'desc',
    'genres', # Selezioniamo la colonna originale
    'Rating'
]]
user_histories_test_df = (
    history_test_data.groupby('user_id')[['title', 'desc', 'genres', 'Rating']]
                .apply(lambda x: x.to_dict('records'))
                .reset_index(name='history')
)

output_filename = datapath + "histories_gemma_recommender_train.json"
print(f"Salvataggio del file JSON in '{output_filename}'...")
user_histories_train_df.to_json(output_filename, orient='records', indent=4, force_ascii=False)

output_filename = datapath + "histories_gemma_recommender_test.json"
print(f"Salvataggio del file JSON in '{output_filename}'...")
user_histories_test_df.to_json(output_filename, orient='records', indent=4, force_ascii=False)


print("✨ Fatto! File JSON creato con successo.")

Creazione delle storie degli utenti...
Salvataggio del file JSON in 'final/histories_gemma_recommender_train.json'...
Salvataggio del file JSON in 'final/histories_gemma_recommender_test.json'...
✨ Fatto! File JSON creato con successo.


In [24]:
merged_test_df = pd.merge(df_with_popularity, app_plots, on='app_id', how='inner')
merged_test_df

,user_id,app_id,Rating,Timestamp,popularity,title,tmdbId,imdbId,desc,genres
0,5950,2550,1,957180825,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
1,1513,2550,3,974749912,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
2,1941,2550,4,974850476,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
3,33,2550,4,978109889,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
4,1733,2550,4,977461820,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
...,...,...,...,...,...,...,...,...,...,...
108018,2167,1720,2,974612960,1,Time Tracers (1995),438108.0,128755.0,When the discovery of a five-thousand year old...,Action|Adventure|Sci-Fi
108019,5333,148,3,989050056,1,"Awfully Big Adventure, An (1995)",22279.0,112427.0,"Liverpool. 1947. Right after World War II, a s...",Drama
108020,2015,2585,1,974677124,1,"Lovers of the Arctic Circle, The (Los Amantes ...",1414.0,133363.0,Otto and Ana are kids when they meet each othe...,Drama|Romance
108021,1812,3540,4,975367635,1,Passion of Mind (1999),19214.0,160644.0,A psychological romantic thriller where fantas...,Romance|Thriller


In [25]:
merged_test_df = pd.merge(df_with_popularity, app_plots, on='app_id', how='inner')
history_test_data = merged_test_df[[
    'user_id',
    'title',
    'desc',
    'genres', # Selezioniamo la colonna originale
    'Rating',
    'popularity'  # Aggiunta della colonna di popolarità
]]
user_histories_test_df = (
    history_test_data.groupby('user_id')[['title', 'desc', 'genres', 'Rating', 'popularity']]
                .apply(lambda x: x.to_dict('records'))
                .reset_index(name='history')
)


output_filename = datapath + "histories_gemma_recommender_test_with_popularity.json"
print(f"Salvataggio del file JSON in '{output_filename}'...")
user_histories_test_df.to_json(output_filename, orient='records', indent=4, force_ascii=False)


print("✨ Fatto! File JSON creato con successo.")

Salvataggio del file JSON in 'tuning/histories_gemma_recommender_test_with_popularity.json'...
✨ Fatto! File JSON creato con successo.


## CANDIDATI

In [13]:
import pandas as pd
datapath = "final/"
candidate_items = pd.read_csv(datapath + "candidate_items_30.csv")
app_plots = pd.read_csv(datapath + "app_plots.csv")
app_plots.columns, candidate_items.columns

(Index(['app_id', 'title', 'genres', 'new_title', 'year', 'titolo_match',
        'plot', 'imdbId', 'tmdbId'],
       dtype='object'),
 Index(['user_id', 'item_id'], dtype='object'))

In [14]:
candidate_items.rename(columns={'item_id': 'app_id'}, inplace=True)
candidate_items

,user_id,app_id
0,1,364
1,1,1198
2,1,1210
3,1,1196
4,1,2858
...,...,...
181195,6040,2731
181196,6040,562
181197,6040,1095
181198,6040,3197


In [15]:
candidate_items = pd.merge(candidate_items, app_plots, on='app_id', how='inner')
candidate_items

,user_id,app_id,title,genres,new_title,year,titolo_match,plot,imdbId,tmdbId
0,1,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0
1,21,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0
2,55,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0
3,58,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0
4,99,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0
...,...,...,...,...,...,...,...,...,...,...
181195,5831,2177,Family Plot (1976),Comedy|Thriller,Family Plot,1976,Family Plot,"A fake psychic, Blanche Tyler (Barbara Harris)...",74512.0,5854.0
181196,5845,3310,"Kid, The (1921)",Action,"Kid, The",1921,"Kid, The",A tramp cares for a boy after he's abandoned a...,12349.0,10098.0
181197,5896,2799,Problem Child 2 (1991),Comedy,Problem Child 2,1991,Problem Child 2,"After Ben's divorce from Flo, he and Junior, m...",102719.0,28597.0
181198,5928,3133,Go West (1925),Comedy,Go West,1925,Go West,"A drifter identified only as ""Friendless"" (Kea...",15863.0,33015.0


In [16]:
candidate_items.to_csv(datapath + "candidate_items_30.csv", index=False)

## fix candidati training

In [14]:
import pandas as pd
import numpy as np
datapath = "final/"
test_recommendations = pd.read_csv(datapath + "test_recommendations.csv")
candidate_items = pd.read_csv(datapath + "candidate_items_30.csv")
app_plots = pd.read_csv(datapath + "app_plots.csv")
test_recommendations = test_recommendations.merge(app_plots, on="app_id", how="left")
def dcg_score(scores):
    return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(len(scores), dtype=np.float64) + 2)))

def ndcg_at_10(ranked_relevance_scores, num_relevant_items):
    rank_dcg = dcg_score(ranked_relevance_scores)
    if rank_dcg == 0.0: return 0.0
    ideal_scores = np.ones(min(num_relevant_items, 10))
    ideal_dcg = dcg_score(ideal_scores)
    if ideal_dcg == 0.0: return 0.0
    return rank_dcg / ideal_dcg

test_recommendations.columns, candidate_items.columns

(Index(['user_id', 'app_id', 'Rating', 'Timestamp', 'title', 'genres',
        'new_title', 'year', 'titolo_match', 'plot', 'imdbId', 'tmdbId'],
       dtype='object'),
 Index(['user_id', 'app_id', 'title', 'genres', 'new_title', 'year',
        'titolo_match', 'plot', 'imdbId', 'tmdbId'],
       dtype='object'))

In [10]:
test_recommendations.drop(columns=['Timestamp', 'Rating'], inplace=True)
candidate_items.columns, test_recommendations.columns

(Index(['user_id', 'app_id', 'title', 'genres', 'new_title', 'year',
        'titolo_match', 'plot', 'imdbId', 'tmdbId'],
       dtype='object'),
 Index(['user_id', 'app_id', 'title', 'genres', 'new_title', 'year',
        'titolo_match', 'plot', 'imdbId', 'tmdbId'],
       dtype='object'))

In [15]:
all_ndcgs = []
results = []
all_hits = []
for user_id in test_recommendations['user_id'].unique():
    rec = candidate_items[candidate_items['user_id'] == user_id]
    real_rec = test_recommendations[test_recommendations['user_id'] == user_id]
    relevance_scores = rec['app_id'].isin(real_rec['app_id']).astype(int).values
    num_relevant_items = real_rec.shape[0]
    ndcg = ndcg_at_10(relevance_scores, num_relevant_items)
    hits = relevance_scores.sum()
    print(f"User {user_id} - NDCG@10: {ndcg:.4f}")
    all_ndcgs.append(ndcg)
    all_hits.append(hits)
    results.append({
        "user_id": user_id,
        "ndcg_score": ndcg,
        "hits": hits
    })
    #controlla se in results c'è un utente con hits = 0
count=0
for res in results:
    if res['hits'] == 0:
        print(f"User {res['user_id']} has 0 hits")
        count+=1
print(f"Total users with 0 hits: {count}")
print(f"Average NDCG@10: {np.mean(all_ndcgs):.4f}")


User 5412 - NDCG@10: 0.4466
User 5440 - NDCG@10: 0.2159
User 368 - NDCG@10: 0.6896
User 425 - NDCG@10: 0.2659
User 4942 - NDCG@10: 0.4897
User 4668 - NDCG@10: 0.2346
User 2907 - NDCG@10: 0.4334
User 2376 - NDCG@10: 0.5824
User 3824 - NDCG@10: 0.6045
User 5605 - NDCG@10: 0.4322
User 4414 - NDCG@10: 0.1700
User 5709 - NDCG@10: 0.9293
User 5812 - NDCG@10: 0.3520
User 285 - NDCG@10: 0.1032
User 5491 - NDCG@10: 0.2915
User 824 - NDCG@10: 0.1258
User 349 - NDCG@10: 0.8510
User 817 - NDCG@10: 0.3354
User 3228 - NDCG@10: 0.4755
User 4626 - NDCG@10: 0.2267
User 5805 - NDCG@10: 0.2606
User 4732 - NDCG@10: 1.1225
User 1800 - NDCG@10: 0.2460
User 3163 - NDCG@10: 0.2344
User 1422 - NDCG@10: 0.4162
User 5172 - NDCG@10: 0.4603
User 2892 - NDCG@10: 0.1570
User 5166 - NDCG@10: 0.3107
User 58 - NDCG@10: 0.5401
User 411 - NDCG@10: 0.3773
User 3272 - NDCG@10: 0.6077
User 2421 - NDCG@10: 0.2945
User 1975 - NDCG@10: 0.1284
User 1966 - NDCG@10: 0.0851
User 5785 - NDCG@10: 0.3768
User 3415 - NDCG@10: 0.1048
U

In [ ]:
import pandas as pd
import numpy as np

# Copia del DataFrame originale per non modificarlo direttamente
candidate_items_fixed = candidate_items.copy()

# Lista utenti senza hit
users_with_zero_hits = []

for user_id in test_recommendations['user_id'].unique():
    rec = candidate_items_fixed[candidate_items_fixed['user_id'] == user_id]
    real_rec = test_recommendations[test_recommendations['user_id'] == user_id]
    
    relevance_scores = rec['app_id'].isin(real_rec['app_id']).astype(int).values
    hits = relevance_scores.sum()
    
    if hits == 0:
        users_with_zero_hits.append(user_id)

print(f"Utenti senza hit: {len(users_with_zero_hits)}")

# Correggi i candidate_items per questi utenti
for user_id in users_with_zero_hits:
    # Candidati attuali
    user_candidates = candidate_items_fixed[candidate_items_fixed['user_id'] == user_id]
    
    if user_candidates.empty:
        continue  # salta se non ci sono candidati (caso limite)

    # Un elemento da rimuovere a caso
    random_candidate_index = np.random.choice(user_candidates.index)
    candidate_items_fixed.drop(index=random_candidate_index, inplace=True)
    
    # Prendi un elemento vero dal test set di quell'utente
    user_test_items = test_recommendations[test_recommendations['user_id'] == user_id]
    user_test_sample = user_test_items.sample(n=1, random_state=42)
    
    # Aggiungilo ai candidate_items_fixed
    candidate_items_fixed = pd.concat([candidate_items_fixed, user_test_sample], ignore_index=True)

print(f"✅ Candidati corretti: ora tutti gli utenti hanno almeno un hit garantito.")
users_with_zero_hits=[]
for user_id in test_recommendations['user_id'].unique():
    rec = candidate_items_fixed[candidate_items_fixed['user_id'] == user_id]
    real_rec = test_recommendations[test_recommendations['user_id'] == user_id]
    
    relevance_scores = rec['app_id'].isin(real_rec['app_id']).astype(int).values
    hits = relevance_scores.sum()
    
    if hits == 0:
        users_with_zero_hits.append(user_id)

print(f"Utenti senza hit: {len(users_with_zero_hits)}")


Utenti senza hit: 585
✅ Candidati corretti: ora tutti gli utenti hanno almeno un hit garantito.
Utenti senza hit: 0


In [13]:
candidate_items_fixed.to_csv(datapath + "candidate_items_30_hits.csv", index=False)

In [22]:
import pandas as pd
datapath = "tuning/"
candidate_items = pd.read_csv(datapath + "candidate_items_30_hits.csv")
movies_with_desc = pd.read_csv(datapath + "movies_with_desc.csv")
candidate_items.columns, movies_with_desc.columns

(Index(['user_id', 'app_id', 'title', 'genres', 'new_title', 'year',
        'titolo_match', 'plot', 'imdbId', 'tmdbId'],
       dtype='object'),
 Index(['app_id', 'title', 'tmdbId', 'imdbId', 'desc', 'genres'], dtype='object'))

In [23]:
merged = candidate_items.merge(
    movies_with_desc[['app_id', 'desc']], 
    on='app_id', 
    how='left'
)

print(merged.head())

   user_id  app_id                  title                        genres  \
0        1     364  Lion King, The (1994)  Animation|Children's|Musical   
1       21     364  Lion King, The (1994)  Animation|Children's|Musical   
2       49     364  Lion King, The (1994)  Animation|Children's|Musical   
3       93     364  Lion King, The (1994)  Animation|Children's|Musical   
4      114     364  Lion King, The (1994)  Animation|Children's|Musical   

        new_title  year    titolo_match  \
0  Lion King, The  1994  Lion King, The   
1  Lion King, The  1994  Lion King, The   
2  Lion King, The  1994  Lion King, The   
3  Lion King, The  1994  Lion King, The   
4  Lion King, The  1994  Lion King, The   

                                                plot    imdbId  tmdbId  \
0  Young lion prince Simba, eager to one day beco...  110357.0  8587.0   
1  Young lion prince Simba, eager to one day beco...  110357.0  8587.0   
2  Young lion prince Simba, eager to one day beco...  110357.0  8587

In [24]:
merged.to_csv(datapath + "candidate_items_30_hits.csv", index=False)

In [25]:
candidate_items_30_hits = pd.read_csv(datapath + "candidate_items_30_hits.csv")
candidate_items_30_hits.head()

,user_id,app_id,title,genres,new_title,year,titolo_match,plot,imdbId,tmdbId,desc
0,1,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
1,21,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
2,49,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
3,93,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
4,114,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
